In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#loading dataset
dataset = datasets.SVHN(root="data", download=True, transform=ToTensor())
test_svhn = datasets.SVHN(root="data", split="test", download=True, transform=ToTensor())

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [3]:
loader = DataLoader(dataset, batch_size=384, shuffle=True)

In [4]:
type(dataset)

torchvision.datasets.svhn.SVHN

In [5]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers1 = nn.Sequential(
            nn.ConvTranspose2d(168, 448, 2, 1, bias=False),
            nn.BatchNorm2d(448),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(448, 256, 4, 2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(256, 128, 4, 2, padding=1, bias=False),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 4, 2, padding=1, bias=False),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(64, 3, 4, 2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.layers1(x)


In [6]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers2 = nn.Sequential(
        nn.Conv2d(1, 256, 4, 2, 1, bias=False),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(256, 512, 4, 2, 1, bias=False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(512, 1024, 3, 2, 0, bias=False),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(1024, 1, 3, 1, 0, bias=False),
    )

    def _weight_init(self):
      for p in self.modules():
        if isinstance(p,nn.Conv2d):
          nn.init.normal_(p.weight, mean=0, std=0.02)
        elif isinstance(p,nn.BatchNorm2d):
          nn.init.normal_(p.weight, mean=1.0, std=0.02)
          nn.init.constant_(p.bias, 0)

    def forward(self,x):
      x=self.layers2(x)
      x=x.view(x.shape[0], -1)
      return x

In [7]:
#2.implementing min_max loss as described in GAN paper
#3.modifying the loss function(Least square) as per LS-GAN paper
#difference in both this type is in discriminator loss where, usual-GAN has BCELoss while LSGAN has MSELoss
#generator loss remains the same, and min_max_loss = discr_loss, gen_loss in both the cases

def discriminator_loss(real,fake,loss_type='GAN'):
  if loss_type == 'GAN':
    criterion = nn.BCELoss(with_logits=True) #2nd part
  else:
    criterion = nn.MSELoss() #3rd part

  real_loss = criterion(real,torch.ones_like(real))
  fake_loss = criterion(fake,torch.zeros_like(fake))

  return real_loss+fake_loss

def generator_loss(fake):
  criterion = nn.BCELoss(with_logits=True)
  return criterion(fake,torch.ones_like(fake))


In [8]:
#hyperparameters
learning_rate = 0.001
epochs = 4

In [9]:
for images in loader:
  print(images[0].shape)
  break

torch.Size([384, 3, 32, 32])


In [10]:
#noise_size = 168*448*2*1 # Example noise vector size, adjust as needed
noise = torch.randn(384*168*448*2).reshape(384,168,448,2)
noise.shape  # Random noise with normal distribution

torch.Size([384, 168, 448, 2])

In [ ]:
#initializing model and optimizers -- for part 2
discriminator = Discriminator()
generator = Generator()

discr_opti = torch.optim.Adam(discriminator.parameters(),lr=learning_rate)
gen_opti = torch.optim.Adam(generator.parameters(),lr=learning_rate)

#4.training on BCE loss -- for part 2
for epoch in range(epochs):
  for i, dataset in enumerate(loader):
    #training discriminator to distinguish between real and fake
    discr_opti.zero_grad()
    real_data = dataset
    fake_data = generator(noise)

    real = discriminator(real_data)
    fake = discriminator(fake_data.detach())

    discr_loss = discriminator_loss(real,fake,loss_type='GAN')
    discr_loss.backward()
    discr_opti.step()

    #training generator to generate fake images close to real ones
    gen_opti.zero_grad()
    fake_data = generator(noise)
    fake = discriminator(fake_data)

    gen_loss = generator_loss(fake)
    gen_loss.backward()
    gen_opti.step()

    if i%100 == 0:
      print(f"[{epoch}/{epochs}][{i}/{len(loader)}]" f"D_Loss: {discr_loss.item():.3f}, G_Loss: {gen_loss.item():.3f}")

In [ ]:
with torch.no_grad():
    fake_images = generator(torch.randn(64, , 1, 1))

plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(fake_images, padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

In [ ]:
#initializing model and optimizers -- part 3
discriminator2 = Discriminator()
generator2 = Generator()

discr_opti2 = torch.optim.Adam(discriminator2.parameters(),lr=learning_rate)
gen_opti2 = torch.optim.Adam(generator2.parameters(),lr=learning_rate)

#4.training on MSE loss -- for part 3
for epoch in range(epochs):
  for i, dataset in enumerate(loader):
    #training discriminator2 to distinguish between real and fake
    discr_opti2.zero_grad()
    real_data = dataset
    fake_data = generator2(noise)

    real = discriminator2(real_data)
    fake = discriminator2(fake_data.detach())

    discr_loss = discriminator2(real,fake,loss_type='LSGAN')
    discr_loss.backward()
    discr_opti2.step()

    #training generator2 to generate fake images close to real ones
    gen_opti2.zero_grad()
    fake_data = generator2(noise)
    fake = discriminator2(fake_data)

    gen_loss = generator2(fake)
    gen_loss.backward()
    gen_opti2.step()

    if i%100 == 0:
      print(f"[{epoch}/{epochs}][{i}/{len(loader)}]" f"D_Loss: {discr_loss.item():.3f}, G_Loss: {gen_loss.item():.3f}")

In [ ]:
with torch.no_grad():
    fake_images = generator2(torch.randn(64, 100, 1, 1))

plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(fake_images, padding=2, normalize=True).cpu(), (1, 2, 0)))
plt.show()

In [14]:
#tried to visualize and generate images for both the parts
#but although my code is running, it keeps on crashing due to RAM usage issue and is unable to train and move forward
#as per my knowledge though, I tried to code task parts - 1,2,3,4 as mentioned in the lab